# Como Fazer "Train Test Split" Corretamente

#### Ao construir modelos de aprendizado de máquina, uma etapa crucial é dividir seus dados em conjuntos de treino, teste e, às vezes, validação. Esta divisão ajuda a garantir que seu modelo tenha um bom desempenho não apenas nos dados que ele já viu, mas também em dados novos e não vistos.

### 1. A Importância do Conjunto de Treino, Teste e Validação
#### 1.1 Conjunto de Treino

Objetivo: Treinar o modelo. 

O conjunto de treino é usado para treinar o modelo, ou seja, ajustar os parâmetros do modelo aos dados. Este é o conjunto em que o modelo "aprende".

#### 1.2 Conjunto de Teste

Objetivo: Avaliar o modelo.

O conjunto de teste é usado para avaliar o desempenho do modelo. Ele fornece uma estimativa imparcial do desempenho do modelo na prática. Se você usar o conjunto de treino para avaliar o desempenho do modelo, poderá superestimar o desempenho do modelo, pois o modelo já viu todos os dados de treinamento.

#### 1.3 Conjunto de Validação

Objetivo: Ajustar hiperparâmetros e selecionar o melhor modelo.

O conjunto de validação é usado para ajustar hiperparâmetros e escolher o melhor modelo entre vários candidatos. Ajuda a evitar o "overfitting" nos dados de treino.

### *O fluxo comum é: Treino --> Validação (opcional) --> Teste*

### 2. Train Test Split Aleatório vs Temporalmente Estruturado
#### 2.1 Aleatório
Este é o método mais comum. Os dados são divididos aleatoriamente em conjuntos de treino e teste. É útil quando as observações são independentes entre si.

Exemplo em Python:

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

X, y = make_classification(n_features=2, n_redundant=0, n_informative=2,
                           n_clusters_per_class=1, n_samples=1000, random_state=42)


# test_size = 0.2 --> significa que 20% dos dados serão usados para teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Total dataset size:")
print("X:", X.shape, "y:", y.shape)
print("Train dataset size:")
print("X:", X_train.shape, "y:", y_train.shape)
print("Test dataset size:")
print("X:", X_test.shape, "y:", y_test.shape)

print("% Train:", X_train.shape[0]/X.shape[0])
print("% Test:", X_test.shape[0]/X.shape[0])

Total dataset size:
X: (1000, 2) y: (1000,)
Train dataset size:
X: (800, 2) y: (800,)
Test dataset size:
X: (200, 2) y: (200,)
% Train: 0.8
% Test: 0.2


#### 2.1 Aleatório com classificação estratificada
Este método é recomendado quando estamos lidando com conjuntos de dados onde a distribuição das classes é desbalanceada. Ele garante que a proporção de cada classe no conjunto de treino e teste seja a mesma que no conjunto de dados original.

Exemplo em Python:

In [31]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Criando um dataset de exemplo desbalanceado
X, y = make_classification(n_features=2, n_redundant=0, n_informative=2,
                           n_clusters_per_class=1, n_samples=1000, weights=[0.999, 0.001], random_state=42)

# Divisão estratificada
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Verificando as proporções
print("Proporção da classe 1 no dataset original:", sum(y) / len(y))
print("Proporção da classe 1 no conjunto de treino:", sum(y_train) / len(y_train))
print("Proporção da classe 1 no conjunto de teste:", sum(y_test) / len(y_test))

Proporção da classe 1 no dataset original: 0.006
Proporção da classe 1 no conjunto de treino: 0.0075
Proporção da classe 1 no conjunto de teste: 0.0


In [32]:
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification

# Criando um dataset de exemplo desbalanceado
X, y = make_classification(n_features=2, n_redundant=0, n_informative=2,
                           n_clusters_per_class=1, n_samples=1000, weights=[0.999, 0.001], random_state=42)

# Divisão estratificada
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

# Verificando as proporções
print("Proporção da classe 1 no dataset original:", sum(y) / len(y))
print("Proporção da classe 1 no conjunto de treino:", sum(y_train) / len(y_train))
print("Proporção da classe 1 no conjunto de teste:", sum(y_test) / len(y_test))

Proporção da classe 1 no dataset original: 0.006
Proporção da classe 1 no conjunto de treino: 0.00625
Proporção da classe 1 no conjunto de teste: 0.005


#### 2.2 Com estrutura temporal
Quando o problema possui estrutura temporal (ou seja, os dados são coletados ao longo do tempo), é importante dividir os dados em conjuntos de treino e teste de acordo com a estrutura temporal. Isso significa que os dados mais antigos são usados para treinar o modelo e os dados mais recentes são usados para testar o modelo. Isso é importante para prevenir o "data leakage" (vazamento de dados), que é quando as informações do futuro são usadas para prever o passado.

Exemplo em Python:

In [9]:
import pandas as pd
import numpy as np

date_rng = pd.date_range(start='2020-01-01', end='2020-12-31', freq='D')
df = pd.DataFrame(date_rng, columns=['date'])
df['data'] = np.random.randint(0,100,size=(len(date_rng)))

# Dividindo os dados
train_size = int(len(df) * 0.8)

# qual foi a data de  corte?
data_corte = df['date'][train_size]

# qual é o index da data de corte?
index_corte = df[df['date'] == data_corte].index[0]

train, test = df[0:index_corte], df[index_corte:len(df)]

print("Total dataset size:", df.shape)
print("Data de corte:", df['date'][train_size])
print("Train dataset size:", train.shape)
print("Test dataset size:", test.shape)
print("% Train:", train.shape[0]/df.shape[0])
print("% Test:", test.shape[0]/df.shape[0])

Total dataset size: (366, 2)
Data de corte: 2020-10-19 00:00:00
Train dataset size: (292, 2)
Test dataset size: (74, 2)
% Train: 0.7978142076502732
% Test: 0.20218579234972678


### 3. Cross Validation
Cross Validation (CV) é uma técnica que divide o conjunto de dados em "k" subconjuntos. O modelo é treinado em "k-1" desses subconjuntos e testado no subconjunto restante. Este processo é repetido "k" vezes, com cada subconjunto usado exatamente uma vez como conjunto de teste.

O método mais comum de CV é o k-fold CV.

Exemplo em Python:

In [33]:
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
scores = cross_val_score(model, X, y, cv=5)

print(f'Scores do Cross Validation: {scores}')


Scores do Cross Validation: [0.995 0.995 0.995 0.995 0.99 ]


### 4. Leakage: vazamento de informação através do pré-processamento
O vazamento de dados ocorre quando informações do conjunto de dados de teste são usadas para treinar o modelo. Isso pode acontecer inadvertidamente durante o pré-processamento. Um exemplo clássico de vazamento é a normalização ou padronização dos dados.

#### 4.1 Normalização
Se normalizarmos (ou padronizarmos) todo o conjunto de dados antes de dividir em treino e teste, estamos usando informações do conjunto de teste para definir a escala dos dados de treino. Isso pode levar a resultados otimistas e enganosos.

Exemplo em Python:

In [57]:
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Criando um dataset com distribuições diferentes para treino e teste
X_train = np.random.randn(800, 1)
X_test = np.random.normal(5, 1, size=(200, 1))
y_train = (X_train > X_train.mean()).astype(int).ravel()
y_test = (X_test > X_test.mean()).astype(int).ravel()

# Concatenando treino e teste
X_combined = np.vstack((X_train, X_test))
y_combined = np.hstack((y_train, y_test))

# Aplicando o StandardScaler de forma incorreta
scaler_bad = StandardScaler()
X_scaled_bad = scaler_bad.fit_transform(X_combined)

X_train_bad, X_test_bad, y_train_bad, y_test_bad = train_test_split(X_scaled_bad, y_combined, test_size=0.2, random_state=42)

# A forma correta
scaler_correct = StandardScaler()
X_train_correct = scaler_correct.fit_transform(X_train)
X_test_correct = scaler_correct.transform(X_test)

# Modelo treinado com a normalização incorreta
model_bad = LogisticRegression(random_state=42)
model_bad.fit(X_train_bad, y_train_bad)
y_pred_bad = model_bad.predict(X_test_bad)

# Modelo treinado com a normalização correta
model_correct = LogisticRegression(random_state=42)
model_correct.fit(X_train_correct, y_train)
y_pred_correct = model_correct.predict(X_test_correct)

# Métricas para a normalização incorreta
accuracy_bad = accuracy_score(y_test_bad, y_pred_bad)
report_bad = classification_report(y_test_bad, y_pred_bad)

# Métricas para a normalização correta
accuracy_correct = accuracy_score(y_test, y_pred_correct)
report_correct = classification_report(y_test, y_pred_correct)

print("---- Resultados com a normalização incorreta ----")
print("Acurácia:", accuracy_bad)
print(report_bad)

print("\n---- Resultados com a normalização correta ----")
print("Acurácia:", accuracy_correct)
print(report_correct)


---- Resultados com a normalização incorreta ----
Acurácia: 0.695
              precision    recall  f1-score   support

           0       0.61      0.80      0.69        85
           1       0.81      0.62      0.70       115

    accuracy                           0.69       200
   macro avg       0.71      0.71      0.69       200
weighted avg       0.72      0.69      0.70       200


---- Resultados com a normalização correta ----
Acurácia: 0.545
              precision    recall  f1-score   support

           0       0.00      0.00      0.00        91
           1       0.55      1.00      0.71       109

    accuracy                           0.55       200
   macro avg       0.27      0.50      0.35       200
weighted avg       0.30      0.55      0.38       200



/Users/josaum/opt/anaconda3/envs/atech-ai/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/josaum/opt/anaconda3/envs/atech-ai/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/josaum/opt/anaconda3/envs/atech-ai/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif